# Physiologically-Based Pharmacokinetic (PBPK) Modeling: Acetaminophen Metabolism and Drug-Drug Interactions

## Laboratory Exercise: Modeling Drug Metabolism and Interactions

This notebook demonstrates a **Physiologically-Based Pharmacokinetic (PBPK)** model of acetaminophen (APAP) metabolism, focusing on how drug-drug interactions can dramatically alter the production of toxic metabolites. We will explore how ethanol and disulfiram interact with the cytochrome P450 enzyme CYP2E1 to affect acetaminophen metabolism.

### Learning Objectives

By the end of this exercise, you will understand:

1. **Multi-scale modeling**: How proteomics (enzyme levels) and metabolomics (metabolite concentrations) interact in biological systems
2. **Enzyme induction**: How ethanol increases CYP2E1 synthesis, potentially increasing toxic metabolite production
3. **Suicide inhibition**: How disulfiram irreversibly inactivates CYP2E1, potentially protecting against toxicity
4. **Emergent behavior**: How competing mechanisms (induction vs. inhibition) interact in complex ways
5. **Network visualization**: How to represent and visualize biochemical reaction networks

### Background: Acetaminophen Toxicity

Acetaminophen (APAP, paracetamol) is a common pain reliever that is generally safe at therapeutic doses. However, overdose can cause severe liver damage through the following mechanism:

1. **Normal metabolism**: Most APAP is safely metabolized via glucuronidation (UGT) or sulfation (SULT) pathways
2. **Toxic pathway**: A small fraction is metabolized by CYP2E1 to form NAPQI (N-acetyl-p-benzoquinone imine), a highly reactive toxic metabolite
3. **Detoxification**: NAPQI is normally detoxified by conjugation with glutathione (GSH)
4. **Toxicity**: When GSH is depleted or NAPQI production is excessive, NAPQI binds to cellular proteins, causing liver damage

### Drug-Drug Interactions

- **Ethanol**: Induces CYP2E1 synthesis, potentially increasing NAPQI production
- **Disulfiram**: Acts as a mechanism-based (suicide) inhibitor, irreversibly inactivating CYP2E1

The interaction between these two drugs creates a complex dynamic where induction and inhibition compete, leading to emergent behavior that cannot be predicted from individual drug effects alone.

---

### Setup: Imports and Data Structures

The following cell loads the required libraries:
- **tellurium**: dynamical modeling (Antimony/SBML)
- **numpy**: numerical arrays
- **matplotlib**: plotting
- **networkx**: graph construction for the reaction network

In [ ]:
# ============================================================================
# IMPORTS
# ============================================================================
import tellurium as te   # PBPK/dynamical modeling (Antimony, SBML)
import numpy as np      # Numerical arrays and simulation results
import matplotlib.pyplot as plt  # Plotting concentration time courses
import networkx as nx   # Reaction network graph (nodes and edges)

print("All imports successful!")

We define the **species** used in the network and in the model:
- **METABOLITES**: APAP, NAPQI, conjugates (APAP_GLC, APAP_SULF, APAP_GSH), and GSH. Each entry can include `name`, `position` (for graph layout), `label` (safe/toxic), and optionally `smiles` for structure drawing.
- **ENZYMES**: UGT1A1, SULT1A1, CYP2E1, GST — with optional UniProt/PDB identifiers.
- **DRUGS**: Quercetin and Probenecid (inhibitors), with SMILES and graph positions.

In [ ]:
# Species used in the reaction network and in Tellurium models.
# 'position' is (x, y) for static graph layout; 'label' drives node coloring.
METABOLITES = [
    {'name': 'APAP', 'smiles': 'CC(=O)Nc1ccc(O)cc1', 'position': (-1, 0), 'label': 'safe'}, 
    {'name': 'NAPQI', 'smiles': 'CC(=O)N=c1ccc(=O)cc1', 'position': (3, 0), 'label': 'toxic'},
    {'name': 'APAP_GSH', 'position': (7, 0), 'label': 'safe', 'smiles': 'CC(=O)NC1=CC=C(C[S]CC[C@H](NC(=O)C[C@H](N)C(=O)O)C(=O)NC[C@H](N)C(=O)O)C=C1'},
    {'name': 'APAP_GLC', 'position': (3, 2), 'label': 'safe', 'smiles': 'CC(=O)NC1=CC=C(O[C@@H]2O[C@H]([C@@H]([C@@H]([C@H]2O)O)O)C(=O)O)C=C1'},
    {'name': 'APAP_SULF', 'position': (3, -2), 'label': 'safe', 'smiles': 'CC(=O)NC1=CC=C(OS(=O)(=O)O)C=C1'},
    {'name': 'GSH', 'position': (5, -1), 'label': 'safe', 'smiles': 'C(CC(=O)N[C@@H](CS)C(=O)NCC(=O)O)[C@@H](C(=O)O)N'}
]
# Enzymes: UGT1A1 (glucuronidation), SULT1A1 (sulfation), CYP2E1 (Phase I), GST (detox).
ENZYMES = [
    {'name': 'CYP2E1', 'uniprot': 'P05181', 'pdb': '3E4E', 'position': (1, 0), 'label': 'enzyme'},
    {'name': 'UGT1A1', 'uniprot': 'P22309', 'position': (1, 1), 'label': 'enzyme'},
    {'name': 'SULT1A1', 'uniprot': 'P50225', 'pdb': '1LS6', 'position': (1, -1), 'label': 'enzyme'},
    {'name': 'GST', 'position': (5, 0), 'label': 'enzyme'}
]
# Drugs that inhibit Phase II enzymes (Quercetin → SULT, Probenecid → UGT).
DRUGS = [
    {'name': 'Quercetin', 'smiles': 'O=C1c3c(O/C(=C1/O)c2ccc(O)c(O)c2)cc(O)cc3O', 'position': (-1, -2), 'label': 'inhibitor'}, 
    {'name': 'Probenecid', 'smiles': 'O=S(=O)(N(CCC)CCC)c1ccc(C(=O)O)cc1', 'position': (-1, 2), 'label': 'inhibitor'}
]

## Part 1: Static Graph of Interactions

Before building the dynamical model, we visualize the **reaction network** we want to model. This static graph shows all the species (nodes) and reactions (edges) in our system.

**Color coding (see legend):**
- **Green**: Safe metabolites (APAP, APAP_GLC, APAP_SULF, APAP_GSH, GSH)
- **Pink**: Toxic metabolite (NAPQI)
- **Blue**: Enzymes (CYP2E1, UGT1A1, SULT1A1, GST)
- **Peach**: Inhibitor drugs (Quercetin, Probenecid)

In [ ]:
def build_reaction_network():
    """
    Builds a directed graph representing the biochemical
    reaction network used in the Tellurium model.
    
    This graph structure helps visualize:
    - Which species interact with each other
    - The direction of reactions (substrate → product)
    - The type of interaction (catalysis, induction, inhibition)
    """
    
    G = nx.DiGraph()
    
    # Add all nodes
    for node in METABOLITES + ENZYMES + DRUGS:
        G.add_node(node['name'])
    
    # --- Reactions / influences
    # APAP metabolism pathway 1
    G.add_edge('APAP', 'CYP2E1', label='N-hydroxylation')
    G.add_edge('CYP2E1', 'NAPQI', label='toxic metabolite')
    G.add_edge('NAPQI', 'GST', label='detoxification')
    G.add_edge('GST', 'APAP_GSH', label='safe metabolite')
    G.add_edge('GSH', 'GST', label='required')

    # APAP metabolism pathway 2
    G.add_edge('APAP', 'UGT1A1', label='glucuronidation')
    G.add_edge('UGT1A1', 'APAP_GLC', label='safe metabolite')

    # APAP metabolism pathway 3
    G.add_edge('APAP', 'SULT1A1', label='sulfonation')
    G.add_edge('SULT1A1', 'APAP_SULF', label='safe metabolite')
        
    # Drug effects on enzyme
    G.add_edge('Quercetin', 'SULT1A1', label='inhibition')
    G.add_edge('Probenecid', 'UGT1A1', label='inhibition')
    
    return G


def draw_reaction_network(G):
    """
    Draws the biochemical reaction network with
    semantic coloring for teaching purposes.
    
    The layout positions:
    - APAP on the left (substrate)
    - NAPQI in the center (toxic metabolite)
    - CYP2E1 at the top (enzyme)
    - Drugs at the top corners (modulators)
    - Detoxification products on the right
    """
    
    pos = {entry['name']: entry['position'] for entry in METABOLITES + ENZYMES + DRUGS}
    
    plt.figure(figsize=(12, 8))
    all_nodes = METABOLITES + ENZYMES + DRUGS
    colors = {
        'toxic': '#ffcccc',      # Light pink/coral for toxic metabolites
        'enzyme': '#cce5ff',      # Light sky blue for enzymes
        'inhibitor': '#ffe5cc',   # Light peach for inhibitors
        'safe': '#ccffcc',        # Light mint green for safe metabolites
    }
    
    for group, color in colors.items():
        nx.draw_networkx_nodes(G, pos,
                               nodelist=[entry['name'] for entry in all_nodes if entry['label'] == group],
                               node_color=color,
                               node_size=2500,
                               label=group)
    
    # Draw edges (reactions)
    nx.draw_networkx_edges(G, pos, arrows=True, arrowstyle='->', width=2, alpha=0.6)
    
    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
    
    # Draw edge labels (reaction types)
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=9)
    
    plt.title('Reaction Network: Metabolism of Acetaminophen and Drug–Drug Interactions', 
              fontsize=14, fontweight='bold')
    plt.legend(scatterpoints=1, loc='upper right', markerscale=0.3, handlelength=1.5, handletextpad=0.5, columnspacing=1.0)
    plt.axis('off')
    plt.tight_layout()
    plt.show()


# Build the graph from METABOLITES/ENZYMES/DRUGS and draw it with colored nodes
print("Building reaction network...")
G = build_reaction_network()
draw_reaction_network(G)
print("Static network visualization complete!")

## Part 2: Visualizing Chemical Compounds

Before modeling the dynamics, let's visually examine the chemical compounds involved in this system. This helps us understand the molecular structures and investigate their similarity visually.

We'll plot all compounds from the METABOLITES and DRUGS lists that have SMILES codes, allowing us to see:
- The structural features of each compound
- How similar or different the compounds are
- The chemical transformations that occur during metabolism

This visual inspection is crucial for understanding the biochemical reactions we're modeling.


In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import MolsToGridImage
from IPython.display import display

# Collect all compounds that have SMILES (from METABOLITES and DRUGS)
compounds_with_smiles = []

# Add metabolites with SMILES
for entry in METABOLITES:
    if 'smiles' in entry:
        compounds_with_smiles.append({
            'name': entry['name'],
            'smiles': entry['smiles'],
            'label': entry.get('label', 'unknown')
        })

# Add drugs with SMILES
for entry in DRUGS:
    if 'smiles' in entry:
        compounds_with_smiles.append({
            'name': entry['name'],
            'smiles': entry['smiles'],
            'label': entry.get('label', 'unknown')
        })

print(f"Found {len(compounds_with_smiles)} compounds with SMILES codes:")
for comp in compounds_with_smiles:
    print(f"  - {comp['name']} ({comp['label']})")

# Parse SMILES to RDKit molecule objects for drawing
molecules = []
legends = []

for comp in compounds_with_smiles:
    try:
        mol = Chem.MolFromSmiles(comp['smiles'])
        if mol is not None:
            molecules.append(mol)
            legends.append(comp['name'])
        else:
            print(f"Warning: Could not parse SMILES for {comp['name']}: {comp['smiles']}")
    except Exception as e:
        print(f"Error parsing {comp['name']}: {e}")

# Draw molecules in a grid with compound names as legends
if molecules:
    # MolsToGridImage signature: (mols, molsPerRow=3, subImgSize=(200,200), legends=None, ...)
    # Pass molecules as first positional argument
    img = MolsToGridImage(
        molecules,  # First positional argument
        molsPerRow=3,
        subImgSize=(300, 300),
        legends=legends
    )
    display(img)
    print(f"\nSuccessfully visualized {len(molecules)} compounds!")
    print(f"Compound names (in order): {', '.join(legends)}")
else:
    print("No valid molecules to display.")


## Part 3: Baseline Model (Phase II Only)

We build a **baseline** dynamical model that includes only **two metabolic pathways** (no toxic NAPQI yet):

1. **Glucuronidation** (UGT1A1): APAP → APAP_GLC  
2. **Sulfation** (SULT1A1): APAP → APAP_SULF  

Enzyme levels are governed by synthesis and degradation fluxes. We define helper functions to load an Antimony model and to simulate and plot selected species over time. The baseline run uses no drugs.


## Part 2: Visualizing Chemical Compounds

**Task:** Before modeling the dynamics, visually examine the chemical compounds involved in this system. Plot all compounds from the `METABOLITES` and `DRUGS` lists that have SMILES codes in a grid, with compound names as legends. This helps understand molecular structures and how they relate to the biochemical reactions we model.

**TODO (step-by-step):**
1. Import RDKit (`Chem`, `MolsToGridImage`) and `IPython.display.display`.
2. Build a list of compounds that have a `'smiles'` key: loop over `METABOLITES` and `DRUGS`, and for each entry with `'smiles'`, append a dict with `name`, `smiles`, and optionally `label`.
3. Parse each SMILES string into an RDKit molecule with `Chem.MolFromSmiles(smiles)`; collect valid molecules and a parallel list of names for legends.
4. If the molecule list is non-empty, call `MolsToGridImage(molecules, molsPerRow=3, subImgSize=(300, 300), legends=legends)` and `display(img)`.
5. Optionally print how many compounds were found and their names.

In [ ]:
# TODO: Part 2 - Visualizing Chemical Compounds
# 1. Import rdkit and display
# 2. Collect compounds with SMILES from METABOLITES and DRUGS
# 3. Parse SMILES to molecules; build lists of molecules and legends
# 4. Draw grid with MolsToGridImage and display(img)

raise NotImplementedError("Replace with your implementation.")

## Part 4: Model with Three Metabolic Pathways

**Task:** Extend the baseline model by adding the **Phase I (toxic) pathway** and **detoxification**. The full model should have three ways APAP is consumed: glucuronidation, sulfation, and formation of NAPQI (which is then detoxified to APAP_GSH). Plot all key species including NAPQI; the red dashed line at y=2 in the plot marks the toxicity threshold (drawn automatically by `simulate_and_plot` when NAPQI is in the species list).

**TODO (step-by-step):**
1. Define a new Antimony string `NAPQI_FORMATION` that adds:
   - **CYP2E1**: synthesis (`J_cyp_syn: -> CYP2E1; k_cyp_syn`), degradation (`J_cyp_deg: CYP2E1 -> ; k_cyp_deg * CYP2E1`), and NAPQI formation (`J_napqi: APAP -> NAPQI; k_cyp * CYP2E1 * APAP`).
   - **Detoxification**: reaction `NAPQI + GSH -> APAP_GSH` (e.g. `J_gst: NAPQI + GSH -> APAP_GSH; k_gst * NAPQI * GSH`), plus GSH synthesis and degradation.
   - Parameters: e.g. `k_cyp_syn`, `k_cyp_deg`, `k_cyp`, `k_gst`, `k_gsh_syn`, `k_gsh_deg` (choose reasonable values).
   - Initial conditions: `APAP_GSH = 0`, `NAPQI = 0`, `CYP2E1 = 1`, `GSH = 1` (and any others required).
2. Build the full model: `MODEL1 = BASE_MODEL + NAPQI_FORMATION`.
3. Load the model with `build_model(MODEL1)`, set `species_to_plot = ['APAP', 'APAP_GLC', 'APAP_SULF', 'NAPQI', 'APAP_GSH', 'GSH']`, and call `simulate_and_plot(model, 'Baseline: No Drugs', t_end=48, species_to_plot=species_to_plot)`.

In [ ]:
# TODO: Part 4 - Model with Three Metabolic Pathways
# 1. Define NAPQI_FORMATION (Phase I + detoxification) in Antimony
# 2. MODEL1 = BASE_MODEL + NAPQI_FORMATION
# 3. build_model(MODEL1), then simulate_and_plot with NAPQI and other species

raise NotImplementedError("Replace with your implementation.")

## Part 5: Drug–Drug Interaction Experiments

**Task:** Add **inhibitors** that reduce Phase II enzyme activity. Less Phase II capacity shunts more APAP through the Phase I (CYP2E1) pathway, increasing NAPQI. Implement three scenarios and compare peak NAPQI and the toxicity threshold (red dashed line) across them.

**Overview:**
1. **Quercetin only**: Inhibits SULT1A1 (sulfation) → higher NAPQI.
2. **Probenecid only**: Inhibits UGT1A1 (glucuronidation) → higher NAPQI.
3. **Quercetin + Probenecid**: Both Phase II pathways inhibited → highest NAPQI.

**TODO (general):**
- Model inhibition by scaling enzyme activity: e.g. `SULT_activity = 1 / (1 + Quercetin / Ki_Q)` and `UGT_activity = 1 / (1 + Probenecid / Ki_P)` in the Antimony model.
- For each experiment: define an Antimony block (initial concentration of the drug(s), inhibition constant(s), and the activity override), build the combined model (BASE + NAPQI_FORMATION + drug block(s)), and call `simulate_and_plot` with the same species list including NAPQI.

### Experiment 1: Quercetin Only

**TODO:**
1. Define `QUERCETIN_MODEL`: set `Quercetin = 10.0`, choose an inhibition constant `Ki_Q` (e.g. 0.01), and set `SULT_activity = 1 / (1 + Quercetin / Ki_Q)`.
2. Build `MODEL2 = BASE_MODEL + NAPQI_FORMATION + QUERCETIN_MODEL`, load it, and run `simulate_and_plot(model, 'Experiment 1: Quercetin Only', t_end=48, species_to_plot=species_to_plot)` with species including NAPQI.

In [ ]:
# TODO: Experiment 1 - Quercetin only
# QUERCETIN_MODEL = """ ... """
# MODEL2 = BASE_MODEL + NAPQI_FORMATION + QUERCETIN_MODEL
# build_model(MODEL2), simulate_and_plot

raise NotImplementedError("Replace with your implementation.")

### Experiment 2: Probenecid Only

**TODO:**
1. Define `PROBENECID_MODEL`: set `Probenecid = 1.0`, choose `Ki_P` (e.g. 0.001), and set `UGT_activity = 1 / (1 + Probenecid / Ki_P)`.
2. Build `MODEL3 = BASE_MODEL + NAPQI_FORMATION + PROBENECID_MODEL`, load it, and run `simulate_and_plot(model, 'Experiment 2: Probenecid Only', ...)`.

In [ ]:
# TODO: Experiment 2 - Probenecid only
# PROBENECID_MODEL = """ ... """
# MODEL3 = BASE_MODEL + NAPQI_FORMATION + PROBENECID_MODEL
# build_model(MODEL3), simulate_and_plot

raise NotImplementedError("Replace with your implementation.")

### Experiment 3: Quercetin + Probenecid

**TODO:**
1. Build the combined model: `MODEL4 = BASE_MODEL + NAPQI_FORMATION + QUERCETIN_MODEL + PROBENECID_MODEL` (reuse the blocks you defined above).
2. Load and run `simulate_and_plot(model, 'Experiment 3: Quercetin + Probenecid', ...)`.
3. Compare peak NAPQI and the toxicity threshold (red dashed line) across all three experiments.

In [ ]:
# TODO: Experiment 3 - Both inhibitors
# MODEL4 = BASE_MODEL + NAPQI_FORMATION + QUERCETIN_MODEL + PROBENECID_MODEL
# build_model(MODEL4), simulate_and_plot

raise NotImplementedError("Replace with your implementation.")

## Summary and Checklist

After completing the notebook, you should have:

- **Part 2**: A grid plot of all compounds (from METABOLITES and DRUGS with SMILES) with RDKit.
- **Part 4**: A full model (Phase II + Phase I + detoxification) and a plot of APAP, conjugates, NAPQI, and GSH with the toxicity threshold line.
- **Part 5**: Three simulation plots (Quercetin only, Probenecid only, both) and a comparison of peak NAPQI and the toxicity threshold across experiments.

**Key concepts:** Phase II inhibition shunts APAP toward CYP2E1 → NAPQI; drug–drug interactions can amplify toxicity risk.

---

**End of Laboratory Exercise**